### A partir da análise de insights, surgiu a seguinte estrutura para o Dashboard:

#### Alocação de recursos por projeto

- Gráfico de barras com valor total e valor solicitado ao CSHG por projeto
- Destacar o número de projetos, o valor total investido, % e total CSHG
- Permitir fazer o sorting por valor CSGH

#### Detalhamento da alocação de recursos

- Distribuição por area de atuação e média de valor por instituição por area
- Tabela com as finanças agregadas e destaque para os outliers
- opcional: area chart

#### Características da instituição

- Número de beneficiados, horas per cápita média, valor por beneficiado
- Scatterplot relacionando per_capita x carga horaria cutoff e quantidade de beneficiados
- (cutoff + per_capita x benefited tradeoff)

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.options.display.max_columns = 200
from sklearn.decomposition import PCA

import plotly.offline as pyoff
import plotly.plotly as py
import plotly.graph_objs as go
from plotly import tools

In [2]:
compfin = pd.read_pickle('compfin.pkl')

### Resource distribution per institution

In [3]:
# restrinjo a analise aos projetos com dados
compfin = compfin[compfin.has_detailed_info]
# adiciono um id
compfin = compfin.assign(id_org = lambda x: x.organizacao + ": " + x.nome_projeto)
compfin.id_org = compfin.id_org.str.strip() + " "
compfin.set_index('id_org', inplace=True)

# ordeno pelo valor total
compfin=compfin.sort_values('valor_total', ascending = True)

In [4]:
trace1 = go.Bar(
                x=(compfin.valor_total/10**6),
                y=compfin.organizacao.str.strip() + " ",
                orientation = 'h',
                text = compfin.nome_projeto.str.strip(),
                name = "Valor Total"
                )

trace2 = go.Bar(
                x=(compfin.solicitado_iCSHG/10**6),
                y=compfin.organizacao.str.strip() + " ",
                orientation = 'h',
                text = compfin.nome_projeto.str.strip(),
                name = "CSHG"
                )

data = [trace1, trace2]

pyoff.init_notebook_mode(connected=True)

#pyoff.iplot({
#    "data": data,
#    "layout": go.Layout(
#                        title="Total de 29 projetos, 37 MM e 22% de Contribuição CSHG", 
#                        barmode = "overlay",
#                        barnorm = "percent",
#                        margin=dict(
#                                    l=220,
#                                    r=10,
#                                    t=140,
#                                    b=80
#                                    )
#                        )
#})

In [6]:
pyoff.plot({
    "data": data,
    "layout": go.Layout(
                        title="Valor total (M) e valor solicitado (M) CSHG por projeto", 
                        colorway=['#00922e', '#014cb1', '#ccb61b', '#954c07', '#47362c'],
        
                        xaxis = dict(hoverformat = '.3f', type = "log"),

                        barmode = "overlay",
                        barnorm = "percent",
                        margin=dict(
                                    l=220,
                                    r=10,
                                    t=140,
                                    b=80
                                    )
                        )
}, auto_open=True)

'file:///home/nocedan/cshg/temp-plot.html'

### Drew down to resource distribution view

In [7]:
consolidated1 = compfin[compfin.valor_total > 0].groupby(['area']).\
nome_projeto.count().to_frame()
consolidated1.rename(columns={"nome_projeto": "counts"}, inplace=True)

consolidated2 = compfin[compfin.valor_total > 0].groupby(['area']).\
solicitado_iCSHG.mean().to_frame()
consolidated2.rename(columns={"solicitado_iCSHG": "means"}, inplace=True)


consolidated3 = compfin[compfin.valor_total > 0].groupby(['area']).\
solicitado_iCSHG.sum().to_frame()
consolidated3.rename(columns={"solicitado_iCSHG": "sums"}, inplace=True)


consolidated = pd.concat([consolidated1,consolidated2,consolidated3], axis=1)
consolidated.reset_index(inplace=True)
consolidated.sort_values('counts', inplace=True)

In [8]:
#consolidated.head()
consolidated.area.str.strip()

4               Gestão de OSCs
0                        Bolsa
2        Educação Complementar
1    Combate à Vulnerabilidade
3        Educação Profissional
Name: area, dtype: object

In [10]:
# fazer esse aqui: https://plot.ly/python/horizontal-bar-charts/

trace1 = go.Scatter(
                y=round(consolidated.means/1000),
                x=consolidated.area.str.strip(),
                text = "Total " + consolidated.area.str.strip() +": "
                    +round(consolidated.sums/1000).astype('str').str.replace('.0',''),
                name = "Solicitado iCSHG",
                marker = dict(size=12))

data = [trace1]

pyoff.init_notebook_mode(connected=True)

pyoff.plot({
    "data": data,
    "layout": go.Layout(title="Média por projeto do valor solicitado CSHG (k) por área",
                        colorway=['#00922e', '#014cb1', '#ccb61b', '#954c07', '#47362c'],
                        yaxis=dict(
                                    title='yaxis title',
                                    rangemode='tozero'
                        ),
                       )
    }, auto_open=True)

'file:///home/nocedan/cshg/temp-plot.html'

In [11]:
trace1 = go.Pie(
                values=consolidated.counts,
                labels=consolidated.area.str.strip(),
                hole = 0.6
                )

data = [trace1]

pyoff.init_notebook_mode(connected=True)

pyoff.plot({
    "data": data,
    "layout": go.Layout(title="Quantidade de projetos por área",
                        #colorway=['#636148', '#92C0FF', '#90AED2', '#CFDFF8', '#9499A4'],
                        #colorway=['#8C4119', '#783012', '#636148', '#AED6B1', '#758D9D'],
                        colorway=['#00922e', '#014cb1', '#ccb61b', '#954c07', '#47362c'],
                       )
}, auto_open=True)

'file:///home/nocedan/cshg/temp-plot.html'

- ou dá um colorido ou muda para linha

In [12]:
compfin_org = compfin.drop_duplicates(subset=['organizacao'], keep='first').copy()
compfin_org.sort_values('receitas_2015', ascending=True, inplace=True)

compfin_org.receitas_2015 = compfin_org.receitas_2015/1000
compfin_org.despesas_2015 = compfin_org.despesas_2015/1000
compfin_org.ativo_2015 = compfin_org.ativo_2015/1000
compfin_org.pl_2015 = compfin_org.pl_2015/1000

In [13]:
compfin_org.head()

,carteira_2016,organizacao,nome_projeto,area,valor_total,beneficiarios,solicitado_iCSHG,carga_horaria,ativo_2014,ativo_2015,pl_2014,pl_2015,receitas_2014,receitas_2015,despesas_2014,despesas_2015,rec_empresas_2015,rec_fundacao_instituto_2015,rec_pf_2015,rec_governo_2015,rec_outros_2015,d_projetos,d_gerais_e_admin,d_folha_de_pagamento,d_outros,per_capita,beneficiarios_iCSHG,per_capita_por_hora,has_detailed_info,has_consolidated_info
id_org,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Saúde Criança / Instituto C: Atuação com Famílias,1,Saúde Criança / Instituto C,Atuação com Famílias,Combate à Vulnerabilidade,364400.0,160.0,224870.0,NaN,587.0,0.900,342.0,0.631,735.0,1.320,587.0,1.031,250980.0,283943.0,416902.0,122774.0,245372.0,639532.0,134902.0,185700.0,71327.0,2277.500000,98.735456,NaN,True,True
Direito FGV: Bolsas Manutenção,2,Direito FGV,Bolsas Manutenção,Bolsa,215098.0,16.0,39000.0,NaN,162.0,1.377,162.0,1.377,221.0,1.495,236.0,0.280,91665.0,0.0,1266596.0,0.0,137029.0,146112.0,18234.0,0.0,116113.0,13443.625000,2.901003,NaN,True,True
PROVE: Projeto Ambiente de Paz,1,PROVE,Projeto Ambiente de Paz,Combate à Vulnerabilidade,314386.0,84.0,314386.0,3048.0,1344.0,1.042,1319.0,1.319,1607.0,1.632,1669.0,1.937,211040.0,0.0,21613.0,0.0,1399526.0,950861.0,420040.0,491293.0,75003.0,3742.690476,84.000000,1.227917,True,True
GMK: Pró-Formação Pró-Morato,2,GMK,Pró-Formação Pró-Morato,Educação Profissional,1009612.0,360.0,360000.0,580.0,8407.0,9.616,8407.0,9.616,706.0,2.625,1399.0,1.416,250800.0,454170.0,11501.0,0.0,1908322.0,1346785.0,66868.0,0.0,2228.0,2804.477778,128.366145,4.835307,True,True
Ser +: Jovem em Destaque,2,Ser +,Jovem em Destaque,Educação Profissional,893735.0,250.0,178680.0,400.0,0.0,NaN,0.0,0.000,5365.0,3.202,5302.0,3.622,NaN,271608.0,630843.0,370468.0,1928583.0,2922502.0,46130.0,570191.0,83075.0,3574.940000,49.981258,8.937350,True,True


In [14]:
trace1 = {"x": compfin_org.receitas_2015, 
          "y": compfin_org.organizacao.str.strip()+" ", 
          "marker": {"size": 12}, 
          "mode": "markers", 
          "name": "Receitas 2015", 
          "type": "scatter",
          "text": compfin_org.receitas_2015.astype('str')
         }

trace2 = {"x": compfin_org.despesas_2015, 
          "y": compfin_org.organizacao.str.strip()+" ", 
          "marker": {"size": 12}, 
          "mode": "markers", 
          "name": "Despesas 2015", 
          "type": "scatter",
          "text": compfin_org.despesas_2015.astype('str')
         }

trace3 = {"x": compfin_org.ativo_2015, 
          "y": compfin_org.organizacao.str.strip()+" ", 
          "marker": {"size": 12}, 
          "mode": "markers", 
          "name": "Ativo 2015", 
          "type": "scatter",
              "text": compfin_org.ativo_2015.astype('str')
         }

trace4 = {"x": compfin_org.pl_2015, 
          "y": compfin_org.organizacao.str.strip()+" ", 
          "marker": {"size": 12}, 
          "mode": "markers", 
          "name": "PL 2015", 
          "type": "scatter",
          "text": compfin_org.pl_2015.astype('str')
         }

data = [trace1, trace2, trace3, trace4]
layout = {"title": "Finanças Consolidadas por Instituição 2015 (M)", 
          "xaxis": {"title": "R$(M)", "type": "log", "hoverformat": '.3f'}, 
          #xaxis = dict(hoverformat = '.3f', type = "log"),
          "yaxis": {"title": "Instituição"},
          "margin": dict(
                                    l=220,
                                    r=10,
                                    t=140,
                                    b=80
                                    ),
          'colorway':['#00922e', '#014cb1', '#ccb61b', '#954c07', '#47362c'],
         }

fig = go.Figure(data=data, layout=layout)

pyoff.plot({
    "data": data,
    "layout": layout
}, auto_open=True)

'file:///home/nocedan/cshg/temp-plot.html'

### Institution assistence characteristics

In [15]:
hover_text = []
bubble_size = []

for index, row in compfin.iterrows():
    hover_text.append(('Organização: {organizacao} <br>'+
                      'Projeto: {nome_projeto} <br>'+
                      'Valor total: {valor_total} <br>').format(organizacao = row['organizacao'],
                               nome_projeto = row['nome_projeto'],
                               valor_total = (row['valor_total']/10**6)))
    
    bubble_size.append(row['beneficiarios'])

compfin['text'] = hover_text
compfin['size'] = bubble_size
sizeref = 1

data = []
for area in set(compfin.area):
    data.append(
        go.Scatter(
        x=compfin['carga_horaria'][compfin['area'] == area],
        y=round(compfin['per_capita'][compfin['area'] == area]),
        mode='markers',
        name=area,
        text=compfin['text'][compfin['area'] == area],
        marker=dict(
            symbol='circle',
            sizemode='area',
            sizeref=1,
            size=compfin['size'][compfin['area'] == area],
            line=dict(
                width=2
                ),
                    )
                )
    )

layout = go.Layout(
    title='Carga horária, valor per capita e beneficiários',
    #colorway=['#00922e', '#014cb1', '#ccb61b', '#954c07', '#47362c'],
    xaxis=dict(
        title='Carga horária (h)',
        gridcolor='rgb(255, 255, 255)',
        #range=[2.003297660701705, 5.191505530708712],
        #type='log',
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    yaxis=dict(
        title='Per capita (R$)',
        gridcolor='rgb(255, 255, 255)',
        #range=[36.12621671352166, 91.72921793264332],
        zerolinewidth=1,
        ticklen=5,
        gridwidth=2,
    ),
    paper_bgcolor='rgb(243, 243, 243)',
    plot_bgcolor='rgb(243, 243, 243)',
)

fig = go.Figure(data=data, layout=layout)

pyoff.plot({
    "data": data,
    "layout": layout
}, auto_open=True)

'file:///home/nocedan/cshg/temp-plot.html'

In [13]:
#hora_per_capita = compfin.carga_horaria/compfin.per_capita
#hora_per_capita = compfin[compfin.index.isin(hora_per_capita.nlargest(5).index)]
#x_= hora_per_capita.carga_horaria 
#y_ = hora_per_capita.per_capita
#annotations = []
#for x_,y_ in zip(x_.values,y_.values):
#    annotations.append(
#            dict(
#            x=x_,
#            y=y_,
#            xref='x',
#            yref='y',
#            text='hr/per_capita',
#            showarrow=True,
#            arrowhead=7,
#            ax=0,
#            ay=-40
#            )
#    )

In [16]:
type(row['valor_total']/10**6)

float

In [17]:
""

''